In [4]:
import pandas as pd
#import pymysql
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
from datetime import datetime, timedelta

load_dotenv()

True

In [5]:
host = os.getenv("HOST")
db_username = os.getenv("DB_USERNAME")
password = os.getenv("PASSWORD")
db_name = os.getenv("LOCAL_DB")
port = os.getenv("PORT")

engine = create_engine(f'mysql+pymysql://root:{password}@{host}/{db_name}')

## create bins for installed capacity

In [6]:
substation_details = pd.read_excel("substation details.xlsx", sheet_name=0, index_col='no')
bins = [0, 40, 80, 160, 320, 1000, 2000]
labels = [1,2,3,4,5,6]
substation_details['capacity_band'] = pd.cut(substation_details['installed_capacity_(mw)'], bins=bins, labels=labels)
substation_details.head()

In [7]:
import numpy as np
substation_details['capacity_band'] = np.where(substation_details['installed_capacity_(mw)']==0,0,substation_details['capacity_band'].astype(int))
substation_details.head()

## Assign codes to the locations

In [22]:
substation_details['region'].value_counts()

central            14
eastern             7
western             6
central western     1
north western       1
north eastern       1
northern            1
Name: region, dtype: int64

## assign codes yo regions depending on their importance 


In [23]:
region_ranks = {"central":5,"eastern":4,"western":3}
def classify_region(row):
    # if key in dict, return value. Else, return 1
    return region_ranks.get(row,2)

In [24]:
substation_details['region_code'] = substation_details['region'].apply(lambda row: classify_region(row))
substation_details['region_code'].value_counts()

5    14
4     7
3     6
2     4
Name: region_code, dtype: int64

In [26]:
substation_details.to_sql(name="substation_details", con=engine, if_exists="replace")

### STORE PROCESSED SUBSTATION DETAILS DATA TO S3

In [27]:
s3_bucket = os.getenv("BUCKET_NAME")
project_folder = os.getenv("GRID_FOLDER")

In [29]:
substation_details = pd.read_sql("SELECT * FROM substation_details", con=engine)
substation_details.head()

,index,no,substation_acronym,substation_name,voltage,voltage_levels,installed_capacity_(mw),region,generators,critical_level,comments,capacity_band,region_code
0,0,1,mskw,masaka west,220/132/33,3,540,central,0,3,supplies masaka city and surroundings,5,5
1,1,2,nkda,nkenda,220/132/33,3,40,western,1,4,IPPs and Hima cement,1,3
2,2,3,owfs,owen falls,132/66/33,3,380,eastern,1,5,evacuates over 380 MW,5,4
3,3,4,homa,hoima,132/33,2,40,western,1,3,supplies hoima city and surroundings. Evacuate...,1,3
4,4,5,ofex,owen falls extension,132/33,2,0,eastern,1,5,200 MW dam connected here,0,4


In [30]:
import awswrangler as wr


wr.s3.to_parquet(
df=substation_details,
path=f's3://{s3_bucket}/{project_folder}/substation_details.parquet',
)


{'paths': ['s3://my-projects-jkimera/grid-project/substation_details.parquet'],
 'partitions_values': {}}